<a href="https://colab.research.google.com/github/smkmohsin/NETFLIX-MOVIES-AND-TV-SHOWS-CLUSTERING/blob/main/NETFLIX_MOVIES_AND_TV_SHOWS_CLUSTERING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem Statement**

This dataset consists of tv shows and movies available on Netflix as of 2019. The dataset is collected from Flixable which is a third-party Netflix search engine.

In 2018, they released an interesting report which shows that the number of TV shows on Netflix has nearly tripled since 2010. The streaming service’s number of movies has decreased by more than 2,000 titles since 2010, while its number of TV shows has nearly tripled. It will be interesting to explore what all other insights can be obtained from the same dataset.

Integrating this dataset with other external datasets such as IMDB ratings, rotten tomatoes can also provide many interesting findings.

## <b>In this  project, you are required to do </b>
1. Exploratory Data Analysis 

2. Understanding what type content is available in different countries

3. Is Netflix has increasingly focusing on TV rather than movies in recent years.
4. Clustering similar content by matching text-based features



# **Attribute Information**

1. show_id : Unique ID for every Movie / Tv Show

2. type : Identifier - A Movie or TV Show

3. title : Title of the Movie / Tv Show

4. director : Director of the Movie

5. cast : Actors involved in the movie / show

6. country : Country where the movie / show was produced

7. date_added : Date it was added on Netflix

8. release_year : Actual Releaseyear of the movie / show

9. rating : TV Rating of the movie / show

10. duration : Total Duration - in minutes or number of seasons

11. listed_in : Genere

12. description: The Summary description

In [275]:
# Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib 
from matplotlib import rcParams

# %matplotlib inline sets the backend of matplotlib to the 'inline' backend
%matplotlib inline

TypeError: ignored

In [276]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [277]:
# Importing the dataset
dataset = pd.read_csv('/content/drive/MyDrive/Almabetter/Module 4/Capstone Project 3/NETFLIX_MOVIES_AND_TV_SHOWS_CLUSTERING.csv')

# <b> Exploratory Data Analysis

In [278]:
dataset.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [279]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   object
 10  listed_in     7787 non-null   object
 11  description   7787 non-null   object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


In [280]:
dataset.describe()

,release_year
count,7787.000000
mean,2013.932580
std,8.757395
min,1925.000000
25%,2013.000000
50%,2017.000000
75%,2018.000000
max,2021.000000


In [281]:
dataset.shape

(7787, 12)

## Data Cleaning

In [282]:
# Missing Value Count Function
def show_missing():
    missing = dataset.columns[dataset.isnull().any()].tolist()
    return missing

# Missing data counts and percentage
print('Missing Data Count')
print(dataset[show_missing()].isnull().sum().sort_values(ascending = False))
print('--'*50)
print('Missing Data Percentage')
print(round(dataset[show_missing()].isnull().sum().sort_values(ascending = False)/len(dataset)*100,2))

Missing Data Count
director      2389
cast           718
country        507
date_added      10
rating           7
dtype: int64
----------------------------------------------------------------------------------------------------
Missing Data Percentage
director      30.68
cast           9.22
country        6.51
date_added     0.13
rating         0.09
dtype: float64


In [283]:
dataset['show_id'].unique()

array(['s1', 's2', 's3', ..., 's7785', 's7786', 's7787'], dtype=object)

In [284]:
dataset['title'].unique()

array(['3%', '7:19', '23:59', ..., 'Zulu Man in Japan',
       "Zumbo's Just Desserts", "ZZ TOP: THAT LITTLE OL' BAND FROM TEXAS"],
      dtype=object)

In [285]:
# Drop unnecessary features
dataset.drop(['director','cast','show_id'], axis=1, inplace=True)

# Drop rows with null value
dataset.dropna(inplace=True)

In [286]:
# After dropping some features and null values
dataset.shape

(7265, 9)

In [287]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7265 entries, 0 to 7786
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          7265 non-null   object
 1   title         7265 non-null   object
 2   country       7265 non-null   object
 3   date_added    7265 non-null   object
 4   release_year  7265 non-null   int64 
 5   rating        7265 non-null   object
 6   duration      7265 non-null   object
 7   listed_in     7265 non-null   object
 8   description   7265 non-null   object
dtypes: int64(1), object(8)
memory usage: 567.6+ KB


In [288]:
dataset.head()

,type,title,country,date_added,release_year,rating,duration,listed_in,description
0,TV Show,3%,Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,Movie,7:19,Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,Movie,23:59,Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,Movie,9,United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,Movie,21,United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


## Top 10 countries producing contents in netflix

In [289]:
# Top 10 countries producing contents in netflix
top_10_country = dataset['country'].value_counts()[:10].reset_index().rename(columns={'index':'country', 'country': 'count'})
top_10_country

,country,count
0,United States,2546
1,India,923
2,United Kingdom,396
3,Japan,224
4,South Korea,183
5,Canada,177
6,Spain,134
7,France,115
8,Egypt,101
9,Turkey,100


In [290]:
plt.pie(top_10_country['count'].values, labels=top_10_country['country'], radius=1.8, startangle=45)
plt.show()

TypeError: ignored

In [ ]:
ratings_count = dataset[['rating','type']].value_counts().reset_index().rename(columns={0: 'count'})
ratings_count

In [ ]:
# Visualize contents based on ratings
sns.barplot('rating', 'count',hue='type',data=ratings_count)
# plt.title('Top 10 countries producing contents in netflix')
rcParams['figure.figsize'] = 10,7

<b><u>Detail about Ratings:</u></b>

**TV-Y:** All Children
Intended for children ages 2 to 6 and is not designed or expected to frighten.

**TV-Y7:** Directed to Older Children
Intended for children ages 7 and older. Best suited for children who know the difference between real life and make-believe. Contains mild fantasy or comedic violence. Some content could frighten younger children (under age 7).

**TV-Y7 FV:** Directed to Older Children - Fantasy Violence
Intended for older children. Contains fantasy violence more combative than TVY7 programs.

**TV-G:** General Audience
Intended for all ages. Contains little or no violence, no strong language and little or no sexual dialogue or situations.

**TV-PG:** Parental Guidance Suggested
Intended for younger children in the company of an adult. Possibly contains some suggestive dialogue, infrequent coarse language, some sexual situations or some moderate violence.

**TV-14:** Parents Strongly Cautioned
Intended for children ages 14 and older in the company of an adult. Possibly contains intensely suggestive dialogue, strong coarse language, intense sexual situations or intense violence.

**TV-MA:** Mature Audience Only
Intended for adults and may be unsuitable for children under 17. Possibly contains crude indecent language, explicit sexual activity or graphic violence.

**G:** General Audiences
This program is designed to be appropriate for all ages. This rating indicates a film contains nothing that would offend parents for viewing by children.

**PG:** Parental Guidance Suggested
Parents are urged to give parental guidance. This film may contain some material parents might not like for their young children.

**PG-13:** Parents Strongly Cautioned
Some material may not be suited for children under age 13. May contain violence, nudity, sensuality, language, adult activities or other elements beyond a PG rating, but doesn’t reach the restricted R category.

**R:** Restricted
This rating is for films specifically designed to be viewed by adults and therefore may be unsuitable for children under 17.

**NC-17:** Clearly Adult
This rating is applied to films the MPAA believes most parents will consider inappropriate for children 17 and under. It indicates only that adult content is more intense than in an R rated movie.

In [ ]:
plt.pie(dataset['type'].value_counts(), labels=['Movies', 'TV Show'], radius=1.8)
plt.show()

## <b>Understanding what type content is available in different countries

In [ ]:
type_of_content =  dataset[['country','type']].value_counts().reset_index().rename(columns={0: 'count'})
type_of_content.head(20)

In [ ]:
# Visualize top 10 countries producing contents in netflix
sns.barplot('country', 'count',hue='type',data=type_of_content[:20])
rcParams['figure.figsize'] = 10, 5

In [ ]:
type_of_content.tail(20)

In [ ]:
# def get_unique_list(col_name, dataset_name):

#   # Extract unique names
#   list_ = []

#   for list_of_name in  dataset_name[col_name].unique():
#     split_list_of_name = list_of_name.split(sep=', ')
#     for name in split_list_of_name:
#       list_.append(name)

#   # Eliminate repeated names
#   list_ = list(set(list_)) 
#   return list_

In [ ]:
# list_of_countries = get_unique_list('country', dataset)

In [ ]:
# # Number of countries producing tv show or movie on netflix
# len(list_of_countries)

In [ ]:
# dataset[dataset.isin({'country': ['United States']})][['country','listed_in']]

In [ ]:
# pd.DataFrame(dataset[dataset['country'] == country]['listed_in'])

In [ ]:
# for country in list_of_countries:
#   print('Type content is available in', country)
#   print('--------------------------------------')
#   print(get_unique_list('listed_in', pd.DataFrame(dataset[dataset['country'] == country]['listed_in'])))
#   print('--------------------------------------')
#   print()
#   print()

## <b>Is Netflix has increasingly focusing on TV rather than movies in recent years.



In [ ]:
type_year_count = dataset[['type', 'release_year']].value_counts().reset_index().rename(columns={0: 'count'})
type_year_count

In [ ]:
sns.lineplot('release_year', 'count', hue='type', data=type_year_count)
rcParams['figure.figsize'] = 20,5

# <b> Clustering similar content by matching text-based features